In [1]:
from chain import summarize

In [2]:
with open('histoire_ff7.txt', 'r') as fichier:
    contenu = fichier.read()
summary = summarize(contenu)
print(summary)

Final Fantasy VII is a role-playing video game developed by Square Enix and released in 1997 on the PlayStation. It is the seventh installment in the Final Fantasy series and the first to use 3D graphics. The game follows the story of an eco-terrorist group called Avalanche fighting against a powerful organization called Shinra. The game was a huge critical and commercial success, selling over 9.7 million copies worldwide. It has had a significant impact on the gaming industry and has inspired a series of sequels, prequels, and a remake. The game's influence can be seen in its use of cinematic storytelling, advanced graphics, and memorable characters like Cloud and Sephiroth.


In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from chain import State, call_model, summarize_history, should_continue, print_update

In [4]:
# Define graph as before:
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_node("conversation", lambda input: call_model(state=input, conversation_summary=summary))
workflow.add_node(summarize_history)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")

# We now add a conditional edge
workflow.add_conditional_edges(
    "conversation",
    should_continue,
)

workflow.add_edge("summarize_history", END)

# Finally, we compile it!
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [7]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="Quels sont les personnages principaux de FF7 ?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)


input_message = HumanMessage(content="Parmi eux qui ont une épée ?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

Quels sont les personnages principaux de FF7 ?
================================== Ai Message ==================================

Les personnages principaux de Final Fantasy VII sont Cloud Strife, Tifa Lockhart, Aerith Gainsborough, Barret Wallace, Red XIII, Cait Sith, Cid Highwind, Vincent Valentine, and Yuffie Kisaragi. Ces personnages jouent des rôles importants dans l'histoire du jeu et ont chacun leurs propres motivations et développements de personnalité.
================================ Human Message =================================

Parmi eux qui ont une épée ?
================================== Ai Message ==================================

Parmi les personnages principaux de Final Fantasy VII, Cloud Strife est le personnage principal qui est le plus associé à une épée emblématique appelée "Buster Sword". Cette épée massive et distinctive est l'arme principale de Cloud et est devenue un symbole e